# Import delle librerie

In [ ]:
import face_recognition
import cv2
import os
import csv
import numpy as np
import pafy
from PIL import Image
import glob
import json
from google import google
import string
import nltk
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Download del video(se necessario)

In [ ]:
# Open the input movie file

url = 'https://www.youtube.com/watch?v=ej3ioOneTy8'
vPafy = pafy.new(url)
video_title = vPafy.title

exists = os.path.isfile(video_title + ".mp4")
if exists:
    video = video_title + ".mp4"
else:
    play = vPafy.getbest()
    video = play.download()

# Import dei face embedding e dei metadati

In [ ]:
import pandas as pd
known_faces = pd.read_csv('athletes_embedding.csv', sep=',',header=None).values
map = pd.read_csv('athletes_metadata.csv', sep=',',header=None).values

# Processamento e riconoscimento di facce da frame del video

In [ ]:
face_locations = []
face_encodings = []
frame_number = 0
face_names = []

input_movie = cv2.VideoCapture(video)
length = int(input_movie.get(cv2.CAP_PROP_FRAME_COUNT))

print("Inizio processamento")
print("L'intero video contiene " + str(length) + " frame.")

while True:
    
    print("Stiamo analizzando il frame numero " + str(frame_number))
    # Grab a single frame of video
    input_movie.set(1, frame_number)
    ret, frame = input_movie.read()
    frame_number += 20

    # Quit when the input video file ends
    if not ret:
        break

    # Find all the faces and face encodings in the current frame of video
    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)


    for face_encoding in face_encodings:
        # See if the face is a match for the known face(s)
        match = face_recognition.compare_faces(known_faces, face_encoding, tolerance=0.50)
        distance = face_recognition.face_distance(known_faces, face_encoding)
        # name = None
        temp_distance = 100
        if True in match:
            for value, count in zip(match, range(len(match))):
                if value == True:
                    if distance[count] < temp_distance:
                        temp_distance = distance[count]
                        name = map[count]
            face_names.append(name[1])
            print("è stata riconosciuta la faccia di " + name[1])

# All done!
input_movie.release()
cv2.destroyAllWindows()
print("Fine")
print (face_names)

# Debug per singolo frame

In [ ]:
frame_number_for_debug = 2360

input_movie = cv2.VideoCapture(video)
input_movie.set(1, frame_number_for_debug)
ret,frame = input_movie.read()

print("Frame estratto dal video:")
imgplot = plt.imshow(frame)
plt.show()
    
face_locations = face_recognition.face_locations(frame)
face_encodings = face_recognition.face_encodings(frame, face_locations)
    
for face_encoding, (top, right, bottom, left) in zip(face_encodings, face_locations):
        # See if the face is a match for the known face(s)
        match = face_recognition.compare_faces(known_faces, face_encoding, tolerance=0.50)
        distance = face_recognition.face_distance(known_faces, face_encoding)
        
        crop_img = frame[top:bottom, left:right]
        imgplot = plt.imshow(crop_img)
        plt.show()
        # name = None
        if True in match:
            for value, count in zip(match, range(len(match))):
                if value == True:
                    print("La distanza rispetto all immagine con indice " + str(count) + " è: " + str(distance[count]))
                    name = map[count]
                    print("è stata riconosciuta la faccia di " + name[1])

# Riduzione e trasformazione in json dell'output

In [ ]:
c = Counter(face_names)
face_names = [x[0] for x in c.items() if x[1] >= 2]

In [ ]:
json.dumps(face_names)

# Interrogare PredictionIO

In [ ]:
#{ "items": ["tt0133240", "tt0153340"], 
#   "num": 4,
#   "categories" : ["Meg Ryan"]}

L'engine di predictionIO ascolta sulla porta 8000. Per conoscere l'ip dell'engine nella rete bridge create, digitare: <br>
    * $> docker inspect --format '{{ .NetworkSettings.IPAddress }}' <name or id container>

# Esempio di default

In [ ]:
import predictionio
#modificare l'ip del server di PredictionIO (guarda il readme in creazione rete)
engine_client = predictionio.EngineClient(url="http://<IP_Engine>:8000")
print(engine_client.send_query({"items": ["EBJWSum8IYQ", "rmjtP94lUTM"], # film simili a questo. E' l'id del video
                                "num": 10, # quanti film voglio che mi raccomandano
                                "tags" : ["neymar", "football"]})) # filtro sugli attori

# Posso mettere anche vari id. Qui ne ho messi due per avere più risultati. Li mette in or.
# Per quanto riguarda l'id di un video, lo si può prendere da:
# https://www.youtube.com/watch?v=<id_video>, per esempio https://www.youtube.com/watch?v=rmjtP94lUTM

# Esempio con il risultato restituito dal face recognizer

In [ ]:
import predictionio
#modificare l'ip del server di PredictionIO (guarda il readme in creazione rete)
engine_client = predictionio.EngineClient(url="http://<IP_Engine>:8000")
print(engine_client.send_query({"items": [id], # film simili a questo. E' l'id di imdb
                                "num": 4, # quanti film voglio che mi raccomandano
                                "tags" : final_list})) # filtro sugli attori